In [1]:
%load_ext autoreload
%autoreload 2
%cd ~/dev/imaginedriving

/home/s0001900/dev/imaginedriving


/home/s0001900/dev/imaginedriving/.venv/lib/python3.10/site-packages/IPython/core/magics/osm.py:393: UserWarning: This is now an optional IPython functionality, using bookmarks requires you to install the `pickleshare` library.
  bkms = self.shell.db.get('bookmarks', {})
/home/s0001900/dev/imaginedriving/.venv/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [6]:
from diffusers import StableDiffusionImg2ImgPipeline, StableDiffusionXLImg2ImgPipeline
import torch

from src.diffusion import ModelId
from src.diffusion import tokenize_prompt
from src.diffusion import encode_prompt

prompt = "hello I like cake"

In [32]:
pipe = StableDiffusionImg2ImgPipeline.from_pretrained(ModelId.sd_v2_1, torch_dtype=torch.float16)
pipe.enable_sequential_cpu_offload()

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

In [31]:
pipe_xl = StableDiffusionXLImg2ImgPipeline.from_pretrained(ModelId.sdxl_base_v1_0, torch_dtype=torch.float16)
pipe_xl.enable_sequential_cpu_offload()

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

In [33]:
tokenizer = pipe.tokenizer
text_encoder = pipe.text_encoder
text_encoder_xl = pipe_xl.text_encoder

In [10]:
tokens = tokenize_prompt(tokenizer, prompt)
tokens

tensor([[49406,  3306,   328,   789,  2972, 49407,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0]])

In [34]:
with torch.no_grad():
    embeds = text_encoder(tokens)
    embeds_xl = text_encoder_xl(tokens)


embeds, embeds_xl

(BaseModelOutputWithPooling(last_hidden_state=tensor([[[-0.3132, -0.4478, -0.0082,  ...,  0.2544, -0.0327, -0.2959],
          [-0.7446,  0.5679,  1.4258,  ...,  1.0156,  0.3279, -0.0911],
          [-0.7915,  0.3477,  1.6562,  ...,  0.9155, -0.7290, -1.4150],
          ...,
          [ 0.3601, -1.7012, -0.1443,  ..., -0.2620, -0.4692,  1.1621],
          [ 0.3335, -1.6807, -0.1266,  ..., -0.5771, -0.5054,  1.2773],
          [ 0.1478, -2.0879, -0.8350,  ..., -0.1511, -0.6709,  1.5195]]],
        dtype=torch.float16), pooler_output=tensor([[ 0.6157,  0.1381,  1.3496,  ..., -0.0580, -0.4976,  2.4688]],
        dtype=torch.float16), hidden_states=None, attentions=None),
 BaseModelOutputWithPooling(last_hidden_state=tensor([[[-0.3884,  0.0229, -0.0523,  ..., -0.4902, -0.3066,  0.0674],
          [ 1.4492,  0.1748, -1.5771,  ..., -0.6709, -0.6450, -0.3672],
          [ 0.6851,  0.7422,  0.9590,  ..., -0.5127, -1.2139, -0.5537],
          ...,
          [ 0.7612, -1.0488, -0.6313,  ..., -1.

In [35]:
def test_encode_same():
    with torch.no_grad():
        assert torch.all(encode_prompt(text_encoder, tokens, False)["embeds"] == text_encoder(tokens)[0])

test_encode_same()